# 数据封装

为了方便用户对数据进行处理，secretflow提供了联邦数据的DataFrame形式封装，DataFrame由多个参与方的数据块组成，支持数据水平和垂直切分两种模式。

<img alt="dataframe.png" src="resource/dataframe.png" width="600">

目前secreflow.DataFrame提供了pandas操作的子集，其使用方式基本与pandas一致。计算过程中，原始数据保持在数据持有方，并不会出域。



下面将演示如何使用DataFrame。

## 前置准备

初始化secretflow，创建两个参与方alice和bob。

In [1]:
import secretflow as sf

sf.init(['alice', 'bob'])
alice = sf.PYU('alice')
bob = sf.PYU('bob')

## 数据准备

这里我们使用[iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) 作为示例数据。

In [2]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
data = pd.concat([iris.data, iris.target], axis=1)
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


我们对数据分别按照水平（特征相同，各持有部分样本）和垂直模式（各持有部分特征）进行切分，便于后续的展示。

In [3]:
# 对Iris数据集进行水平切分
h_alice, h_bob = data.iloc[:70, :], data.iloc[70:, :]

# 保存数据集到文件
import tempfile

_, h_alice_path = tempfile.mkstemp()
_, h_bob_path = tempfile.mkstemp()
h_alice.to_csv(h_alice_path, index=False)
h_bob.to_csv(h_bob_path, index=False)

In [4]:
h_alice

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
65,6.7,3.1,4.4,1.4,1
66,5.6,3.0,4.5,1.5,1
67,5.8,2.7,4.1,1.0,1
68,6.2,2.2,4.5,1.5,1


In [5]:
h_bob

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
70,5.9,3.2,4.8,1.8,1
71,6.1,2.8,4.0,1.3,1
72,6.3,2.5,4.9,1.5,1
73,6.1,2.8,4.7,1.2,1
74,6.4,2.9,4.3,1.3,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
# 对Iris数据集进行垂直切分
v_alice, v_bob = data.iloc[:, :2], data.iloc[:, 2:]

# 保存数据集到文件
_, v_alice_path = tempfile.mkstemp()
_, v_bob_path = tempfile.mkstemp()
v_alice.to_csv(v_alice_path, index=False)
v_bob.to_csv(v_bob_path, index=False)

In [7]:
v_alice

,sepal length (cm),sepal width (cm)
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
...,...,...
145,6.7,3.0
146,6.3,2.5
147,6.5,3.0
148,6.2,3.4


In [8]:
v_bob

,petal length (cm),petal width (cm),target
0,1.4,0.2,0
1,1.4,0.2,0
2,1.3,0.2,0
3,1.5,0.2,0
4,1.4,0.2,0
...,...,...,...
145,5.2,2.3,2
146,5.0,1.9,2
147,5.2,2.0,2
148,5.4,2.3,2


## 创建

### 水平DataFrame

创建一个由水平切分数据组成的DataFrame。

> 💡 原始数据仍然保存在数据持有方本地，没有传输出域。

这里为了方便演示，我们选择了明文聚合和比较。您可以参考[安全聚合](./Secure_aggregation.ipynb)，了解更多安全聚合方案，并根据您的需求施行合适的安全策略。

In [9]:
from secretflow.data.horizontal import read_csv as h_read_csv
from secretflow.security.aggregation import PlainAggregator
from secretflow.security.compare import PlainComparator

# aggregator、comparator分别用于后续的数据分析操作中对数据进行聚合或比较。
hdf = h_read_csv({alice: h_alice_path, bob: h_bob_path}, 
                 aggregator=PlainAggregator(alice), 
                 comparator=PlainComparator(alice))

### 垂直DataFrame

创建一个由垂直切分数据组成的DataFrame。

> 💡 原始数据仍然保存在数据持有方本地，没有传输出域。

In [10]:
from secretflow.data.vertical import read_csv as v_read_csv
from secretflow.security.aggregation import PlainAggregator
from secretflow.security.compare import PlainComparator

vdf = v_read_csv({alice: v_alice_path, bob: v_bob_path})

## 数据分析

出于数据隐私保护的目的，DataFrame不允许查看原始数据。DataFrame提供了和pandas类似的接口便于用户分析数据，这些接口通常同时支持水平和垂直切分数据。

> 💡 以下操作过程中，DataFrame的原始数据仍然保存在节点本地，没有传输出域。

In [11]:
hdf.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [12]:
vdf.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

查看最小值，可以看到和原始数据一致。

In [13]:
print('Horizontal df:\n', hdf.min())
print('\nVertical df:\n', vdf.min())
print('\nPandas:\n', data.min())

Horizontal df:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64

Vertical df:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64

Pandas:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64


同样也可以查看最大值、均值、数量等信息。

In [14]:
hdf.max()

sepal length (cm)    7.9
sepal width (cm)     4.4
petal length (cm)    6.9
petal width (cm)     2.5
target               2.0
dtype: float64

In [15]:
vdf.max()

sepal length (cm)    7.9
sepal width (cm)     4.4
petal length (cm)    6.9
petal width (cm)     2.5
target               2.0
dtype: float64

In [16]:
hdf.mean(numeric_only=True)

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
target               1.000000
dtype: float64

In [17]:
vdf.mean(numeric_only=True)

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
target               1.000000
dtype: float64

In [18]:
hdf.count()

sepal length (cm)    150
sepal width (cm)     150
petal length (cm)    150
petal width (cm)     150
target               150
dtype: int64

In [19]:
vdf.count()

sepal length (cm)    150
sepal width (cm)     150
petal length (cm)    150
petal width (cm)     150
target               150
dtype: int64

### 筛选

获取部分列

In [20]:
hdf_part = hdf[['sepal length (cm)', 'target']]
hdf_part.mean(numeric_only=True)

sepal length (cm)    5.843333
target               1.000000
dtype: float64

In [21]:
vdf_part = hdf[['sepal width (cm)', 'target']]
vdf_part.mean(numeric_only=True)

sepal width (cm)    3.057333
target              1.000000
dtype: float64

### 修改

水平DataFrame

In [22]:
hdf_copy = hdf.copy()
print('Min of target: ', hdf_copy['target'].min()[0])
print('Max of target: ', hdf_copy['target'].max()[0])

Min of target:  0
Max of target:  2


In [23]:
# 将target列的值设为1。
hdf_copy['target'] = 1

# 查看修改后的效果，可以看到target的值都变成1。
print('Min of target: ', hdf_copy['target'].min()[0])
print('Max of target: ', hdf_copy['target'].max()[0])

Min of target:  1
Max of target:  1


垂直DataFrame

In [24]:
vdf_copy = vdf.copy()
print('Min of sepal width (cm): ', vdf_copy['sepal width (cm)'].min()[0])
print('Max of sepal width (cm): ', vdf_copy['sepal width (cm)'].max()[0])

Min of sepal width (cm):  2.0
Max of sepal width (cm):  4.4


In [25]:
# 将sepal width (cm)列的值设为1。
vdf_copy['sepal width (cm)'] = 20

# 查看修改后的效果，可以看到target的值都变成20。
print('Min of sepal width (cm): ', vdf_copy['sepal width (cm)'].min()[0])
print('Max of sepal width (cm): ', vdf_copy['sepal width (cm)'].max()[0])

Min of sepal width (cm):  20
Max of sepal width (cm):  20


## 收尾

In [26]:
# 清理临时文件

import os

try:
    os.remove(h_alice_path)
    os.remove(h_bob_path)
except OSError:
    pass

try:
    os.remove(v_alice_path)
    os.remove(v_bob_path)
except OSError:
    pass